In [136]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge,Lasso,LinearRegression
from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, RobustScaler
from sklearn.model_selection import GridSearchCV

In [137]:
train = pd.read_csv("./data/train.csv")
#train = train.sample(frac=1).reset_index(drop=True)
x_train = train.loc[:,'x1':'x5'].copy()
y_train = train['y'].copy()
x_train = pd.concat([x_train,x_train**2,x_train.apply(np.exp,axis=1),x_train.apply(np.cos,axis=1),
                     pd.Series(np.ones(700))],axis=1)
x_train.columns = np.arange(1,22)

In [138]:
scaler = RobustScaler()
scaler.fit(x_train)
x_train = pd.DataFrame(scaler.transform(x_train))
x_train

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,0.529412,0.6875,-0.4,-0.526316,-0.6,-0.391304,-0.233333,0.401786,0.703704,7.090909e-01,...,0.725963,-0.395881,-0.482902,-0.593990,0.392036,0.233656,-0.401829,-0.697954,-0.708996,0.0
1,-0.352941,1.0625,-0.3,0.210526,0.4,0.160535,0.166667,0.250000,-0.229630,-2.909091e-01,...,1.156921,-0.298391,0.207094,0.406010,-0.160614,-0.166764,-0.250063,0.229689,0.291004,0.0
2,0.882353,0.7500,-0.2,-0.421053,0.4,-0.190635,-0.187500,0.116071,0.548148,-2.909091e-01,...,0.796006,-0.199920,-0.390146,0.406010,0.190896,0.187742,-0.116115,-0.544437,0.291004,0.0
3,0.529412,-0.3750,0.6,-0.526316,0.6,-0.391304,0.262500,-0.312500,0.703704,-3.818182e-01,...,-0.363584,0.624320,-0.482902,0.612080,0.392036,-0.262603,0.312742,-0.697954,0.381958,0.0
4,-0.411765,-0.3750,-0.3,1.631579,-0.6,0.250836,0.262500,0.250000,-1.004630,7.090909e-01,...,-0.363584,-0.298391,1.844200,-0.593990,-0.250903,-0.262603,-0.250063,1.011969,-0.708996,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,2.235294,-0.1875,0.9,1.263158,1.6,2.809365,0.000000,-0.178571,-0.933333,-2.909091e-01,...,-0.184519,0.950811,1.376453,1.673917,-2.792228,0.000000,0.178688,0.939545,0.291004,0.0
696,-1.117647,0.3125,0.7,-0.368421,2.0,1.856187,-0.333333,-0.285714,0.473148,-2.128979e-15,...,0.320103,0.732064,-0.343067,2.113742,-1.849260,0.333862,0.285929,-0.470265,0.000000,0.0
697,-1.176471,-1.0000,0.4,0.052632,0.0,2.033445,1.679167,-0.312500,-0.060185,0.000000e+00,...,-0.923116,0.412038,0.051000,0.000000,-2.024960,-1.675067,0.312742,0.060108,0.000000,0.0
698,1.529412,1.0625,0.0,0.315789,0.2,0.802676,0.166667,-0.098214,-0.333333,-1.636364e-01,...,1.156921,0.000000,0.313789,0.201990,-0.801785,-0.166764,0.098271,0.333734,0.163680,0.0


In [139]:
x_train

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,0.529412,0.6875,-0.4,-0.526316,-0.6,-0.391304,-0.233333,0.401786,0.703704,7.090909e-01,...,0.725963,-0.395881,-0.482902,-0.593990,0.392036,0.233656,-0.401829,-0.697954,-0.708996,0.0
1,-0.352941,1.0625,-0.3,0.210526,0.4,0.160535,0.166667,0.250000,-0.229630,-2.909091e-01,...,1.156921,-0.298391,0.207094,0.406010,-0.160614,-0.166764,-0.250063,0.229689,0.291004,0.0
2,0.882353,0.7500,-0.2,-0.421053,0.4,-0.190635,-0.187500,0.116071,0.548148,-2.909091e-01,...,0.796006,-0.199920,-0.390146,0.406010,0.190896,0.187742,-0.116115,-0.544437,0.291004,0.0
3,0.529412,-0.3750,0.6,-0.526316,0.6,-0.391304,0.262500,-0.312500,0.703704,-3.818182e-01,...,-0.363584,0.624320,-0.482902,0.612080,0.392036,-0.262603,0.312742,-0.697954,0.381958,0.0
4,-0.411765,-0.3750,-0.3,1.631579,-0.6,0.250836,0.262500,0.250000,-1.004630,7.090909e-01,...,-0.363584,-0.298391,1.844200,-0.593990,-0.250903,-0.262603,-0.250063,1.011969,-0.708996,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,2.235294,-0.1875,0.9,1.263158,1.6,2.809365,0.000000,-0.178571,-0.933333,-2.909091e-01,...,-0.184519,0.950811,1.376453,1.673917,-2.792228,0.000000,0.178688,0.939545,0.291004,0.0
696,-1.117647,0.3125,0.7,-0.368421,2.0,1.856187,-0.333333,-0.285714,0.473148,-2.128979e-15,...,0.320103,0.732064,-0.343067,2.113742,-1.849260,0.333862,0.285929,-0.470265,0.000000,0.0
697,-1.176471,-1.0000,0.4,0.052632,0.0,2.033445,1.679167,-0.312500,-0.060185,0.000000e+00,...,-0.923116,0.412038,0.051000,0.000000,-2.024960,-1.675067,0.312742,0.060108,0.000000,0.0
698,1.529412,1.0625,0.0,0.315789,0.2,0.802676,0.166667,-0.098214,-0.333333,-1.636364e-01,...,1.156921,0.000000,0.313789,0.201990,-0.801785,-0.166764,0.098271,0.333734,0.163680,0.0


In [140]:
lm = LinearRegression().fit(x_train,y_train)
lm.coef_

array([  44.55651785,  -61.49471433,  184.47981538,  -61.11918484,
        142.70059208,  -58.7612553 ,    2.16503279,  -42.46086618,
         59.82404539,  -26.80307065,  -41.04766037,   57.02746483,
       -173.21263902,   45.7716348 , -134.72083345,  -62.83594182,
          6.99553086,  -52.41410569,   75.28570077,  -34.3242448 ,
          0.        ])

In [143]:
params = {'alpha': np.arange(0.01,1e2,0.1)}

#est = Ridge(fit_intercept=False,max_iter=1000)
est = Lasso(fit_intercept=False,max_iter=1e5)
#clf = GridSearchCV(est,param_grid=params,cv=10,scoring="neg_mean_squared_error")
clf = GridSearchCV(est,param_grid=params,cv=10,scoring="r2")
clf.fit(x_train, y_train)

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=False,
                             max_iter=100000.0, normalize=False, positive=False,
                             precompute=False, random_state=None,
                             selection='cyclic', tol=0.0001, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'alpha': array([1.000e-02, 1.100e-01, 2.100e-01, 3.100e-01, 4.100e-01, 5.100e-01...
       9.721e+01, 9.731e+01, 9.741e+01, 9.751e+01, 9.761e+01, 9.771e+01,
       9.781e+01, 9.791e+01, 9.801e+01, 9.811e+01, 9.821e+01, 9.831e+01,
       9.841e+01, 9.851e+01, 9.861e+01, 9.871e+01, 9.881e+01, 9.891e+01,
       9.901e+01, 9.911e+01, 9.921e+01, 9.931e+01, 9.941e+01, 9.951e+01,
       9.961e+01, 9.971e+01, 9.981e+01, 9.991e+01])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='r2', verbose=0)

In [144]:
print(clf.best_estimator_)
print(clf.best_score_)
print(clf.best_estimator_.coef_)

Lasso(alpha=0.11, copy_X=True, fit_intercept=False, max_iter=100000.0,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)
-0.04187014591361981
[ 0.         -0.          0.          0.          0.          0.
  0.          0.11684386  0.          0.          0.00404779 -0.
  0.14765646  0.6882764   0.23397729 -0.         -0.         -0.
 -0.         -0.1074525   0.        ]


In [146]:
coefs = np.append(np.zeros((1,20)), np.mean(y_train))
np.savetxt("./data/res_farzam.csv",coefs,delimiter=",")

#Description

In the above code, first I tried OLS model, which did not yield very good results. Then I started to look at features and how useful they are. Interestingy, regardless of the Rigde or Lasso method, the R2 metric was always negetive, which means the model is worse than just predicting the average Y. I also used standard and robust scaler to make sure these effect are not due to the scaling. Additionally, I shuffled the data to avoid systematic errors in recording data, still the R2 measure was negative. Hence, the assumption is that all coefficients are zero with mean(y) the only predi